In [1]:
## Deepinsight Python
import pandas as pd
import time
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.spatial import ConvexHull
import numpy as np
from sklearn.preprocessing import MinMaxScaler
import math
from scipy.spatial import distance
from statistics import mean
from PIL import Image
from matplotlib import cm
import os
import natsort


In [2]:
#list_dir = os.listdir('plots_515/umap_5_0.1')
#list_dir = os.listdir('plots_515/umap/data_finaldf')
#list_dir = natsort.natsorted(list_dir)

clinical_mrna = pd.read_csv("data/clinical_data_subsets/clinical_subset_mirna.csv")
#clinical_meth = pd.read_csv("data/clinical_data_subsets/clinical_subset_meth.csv")
patient_id = clinical_mrna["sample"].values
status = clinical_mrna["vital_status"].values

#data = pd.read_csv("data/clean_datasets/mrna_clean.csv")
data = pd.read_csv("data/clean_datasets/mirna_clean.csv")
#data1 = pd.read_csv("data/mrna576.csv", sep = '\t')

In [4]:
del data["Unnamed: 0"]
data

,Gene,TCGA-05-4384-01,TCGA-05-4390-01,TCGA-05-4396-01,TCGA-05-4405-01,TCGA-05-4410-01,TCGA-05-4415-01,TCGA-05-4417-01,TCGA-05-4424-01,TCGA-05-4425-01,...,TCGA-NJ-A4YG-01,TCGA-NJ-A4YI-01,TCGA-NJ-A4YP-01,TCGA-NJ-A4YQ-01,TCGA-NJ-A55A-01,TCGA-NJ-A55O-01,TCGA-NJ-A55R-01,TCGA-NJ-A7XG-01,TCGA-O1-A52J-01,TCGA-S2-AA1A-01
0,A1BG,0.103785,0.390212,0.556166,0.220240,0.109822,0.214732,0.407931,0.097488,0.385700,...,0.126741,0.099637,0.202561,0.108977,0.159036,0.221377,0.095410,0.098642,0.077353,0.087715
1,A2BP1,0.421911,0.716803,0.752013,0.708389,0.683568,0.638347,0.748030,0.406066,0.807649,...,0.657037,0.457161,0.508750,0.536928,0.739694,0.650683,0.724803,0.789757,0.340416,0.706356
2,A2LD1,0.811461,0.868074,0.786287,0.793372,0.794328,0.860457,0.867543,0.842789,0.849442,...,0.705310,0.794955,0.807477,0.810726,0.740477,0.781828,0.843660,0.744699,0.770962,0.791469
3,A2M,0.690764,0.472345,0.814020,0.735478,0.839948,0.899105,0.698195,0.580405,0.934962,...,0.738193,0.619270,0.859352,0.694127,0.733371,0.773741,0.644528,0.815886,0.710859,0.754945
4,A2ML1,0.713443,0.877008,0.518714,0.851672,0.658261,0.843823,0.809481,0.707396,0.820028,...,0.839704,0.854999,0.623570,0.803369,0.726458,0.802312,0.829503,0.510710,0.874853,0.834402
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17047,ZYG11B,0.057656,0.100332,0.101417,0.076262,0.063710,0.078837,0.081917,0.097576,0.078855,...,0.102317,0.092981,0.100377,0.080889,0.087874,0.060100,0.076161,0.071369,0.057753,0.063034
17048,ZYX,0.525542,0.642904,0.476003,0.663492,0.637852,0.564296,0.505278,0.624332,0.329121,...,0.583890,0.612071,0.683062,0.449519,0.545217,0.579040,0.509494,0.584858,0.659536,0.595179
17049,ZZEF1,0.469237,0.514640,0.493650,0.903876,0.870210,0.890036,0.692273,0.819277,0.876575,...,0.777293,0.843229,0.683032,0.829333,0.851191,0.663383,0.600901,0.336140,0.836211,0.606730
17050,ZZZ3,0.114871,0.081276,0.082344,0.067287,0.080240,0.067297,0.052242,0.063264,0.080818,...,0.143320,0.093946,0.068768,0.153817,0.118883,0.085743,0.117010,0.047989,0.157502,0.052036


In [ ]:
# Dont run for singal tsne/umap output
list_dir = os.listdir('tsne_515/tsne_6_cosine/data_finaldf')
list_dir = natsort.natsorted(list_dir)

In [ ]:
for datafile in list_dir:
    df_subset = pd.read_csv("plots_515/tsne_6_cosine/data_finaldf/" + datafile) #tsne output
    #df_subset = pd.read_csv("plots_515/umap/data_finaldf/umap_5_10.0.csv") #umap output
    #df_subset = pd.read_csv("plots_515/tsne/data_finaldf/tsne_6___cosine.csv") # tsne_out
    df_subset = df_subset[['a', 'b']]
    print("Datafile:", datafile)

    def minimum_bounding_rectangle(points):
        """
        Find the smallest bounding rectangle for a set of points.
        Returns a set of points representing the corners of the bounding box.

        :param points: an nx2 matrix of coordinates
        :rval: an nx2 matrix of coordinates
        """
        from scipy.ndimage.interpolation import rotate
        pi2 = np.pi/2.

        # get the convex hull for the points
        hull_points = points[ConvexHull(points).vertices]

        # calculate edge angles
        edges = np.zeros((len(hull_points)-1, 2))
        edges = hull_points[1:] - hull_points[:-1]

        angles = np.zeros((len(edges)))
        angles = np.arctan2(edges[:, 1], edges[:, 0])

        angles = np.abs(np.mod(angles, pi2))
        angles = np.unique(angles)

        # find rotation matrices
        # XXX both work
        rotations = np.vstack([
            np.cos(angles),
            np.cos(angles-pi2),
            np.cos(angles+pi2),
            np.cos(angles)]).T
        rotations = rotations.reshape((-1, 2, 2))

        # apply rotations to the hull
        rot_points = np.dot(rotations, hull_points.T)

        # find the bounding points
        min_x = np.nanmin(rot_points[:, 0], axis=1)
        max_x = np.nanmax(rot_points[:, 0], axis=1)
        min_y = np.nanmin(rot_points[:, 1], axis=1)
        max_y = np.nanmax(rot_points[:, 1], axis=1)

        # find the box with the best area
        areas = (max_x - min_x) * (max_y - min_y)
        best_idx = np.argmin(areas)

        # return the best box
        x1 = max_x[best_idx]
        x2 = min_x[best_idx]
        y1 = max_y[best_idx]
        y2 = min_y[best_idx]
        r = rotations[best_idx]

        rval = np.zeros((4, 2))
        rval[0] = np.dot([x1, y2], r)
        rval[1] = np.dot([x2, y2], r)
        rval[2] = np.dot([x2, y1], r)
        rval[3] = np.dot([x1, y1], r)

        return rval

    mbr = minimum_bounding_rectangle(df_subset.values)

    # Calculate tilit of bounding box
    y2,y1,x2,x1 = mbr[0][1],mbr[1][1],mbr[0][0],mbr[1][0]
    theta = (y2-y1)/(x2-x1)
    angle  = math.degrees(np.arctan(theta))
    angle  = np.arctan(theta)
    r_matrix = np.asarray([[math.cos(angle),-1*math.sin(angle)],[math.sin(angle),math.cos(angle)]])

    zrect = np.matmul(mbr,r_matrix)  
    z = np.asarray(np.matmul(df_subset,r_matrix))
    rz_subset  = pd.DataFrame()
    rz_subset['rot-tsne-2d-one'] = z[:,0]
    rz_subset['rot-tsne-2d-two'] = z[:,1]
    rz_subset.to_csv('tsne_images/mrna515_tsne_6_cosine_100itr/mrna515_6_cosine_100itr_rotated.csv')

    z_dist = distance.cdist(z, z, 'euclidean')
    min_z_dist = z_dist[z_dist>0].min()

    rec_x_axis,rec_y_axis = abs(zrect[0][0] - zrect[1][0]),abs(zrect[1][1] - zrect[2][1])

    precision_old = math.sqrt(2)
    A = math.ceil((rec_x_axis*precision_old)/min_z_dist)
    B = math.ceil((rec_y_axis*precision_old)/min_z_dist)

    max_pix_size = 120
    if (max([A,B]) > max_pix_size):
        precision = (precision_old*max_pix_size/max([A,B]))
        A = math.ceil((rec_x_axis*precision)/min_z_dist)
        B = math.ceil((rec_y_axis*precision)/min_z_dist)

    x_coord,y_coord = rz_subset.iloc[:,0].values,rz_subset.iloc[:,1].values
    x_min,y_min,x_max,y_max = min(x_coord),min(y_coord),max(x_coord),max(x_coord)
    x_pixel = (1 + (A*(x_coord - x_min))/(x_max - x_min))
    y_pixel = (1 + (B*(y_coord - y_min))/(y_max - y_min))
    round_x_pixel = np.array([int(np.round(x)) for x in x_pixel])
    round_y_pixel = np.array([int(np.round(y)) for y in y_pixel])

    pix_subset  = pd.DataFrame()
    pix_subset['pix-tsne-2d-one'] = round_x_pixel
    pix_subset['pix-tsne-2d-two'] = round_y_pixel
    pix_subset.to_csv('tsne_images/mrna515_tsne_6_cosine_100itr/mrna51_6_cosine_100itr_pix_coord.csv')

    #function to find overlapping points
    def list_duplicates_of(seq,item):
        start_at = -1
        locs = []
        while True:
            try:
                loc = seq.index(item,start_at+1)
            except ValueError:
                break
            else:
                locs.append(loc)
                start_at = loc
        return locs

    #********************************For full image**************************
    pixel_set = []
    for _ in range(len(round_x_pixel)):
        pixel_set.append((round_x_pixel[_],round_y_pixel[_]))

    unique_coord = {}
    for i in range(len(pixel_set)):
        unique_coord[pixel_set[i]] = list_duplicates_of(pixel_set,pixel_set[i])

    clean_data = data.iloc[:,1:].T
    mmscaler = MinMaxScaler()
    #mmscaler.fit(clean_data)
    norm_data = pd.DataFrame(mmscaler.fit_transform(clean_data))
    sample_no = 0
    fvec = norm_data.iloc[sample_no,:]

    new_A = max(round_x_pixel)
    new_B = max(round_y_pixel)

    im_size =  max(new_A, new_B)

    for i in range(norm_data.shape[0]):
        sample_no = i
        imgM = np.zeros((im_size+1,im_size+1))
        fvec = norm_data.iloc[sample_no,:]
        for j in range(norm_data.shape[1]):
            imgM[round_x_pixel[j]-1,round_y_pixel[j]-1] = mean([fvec[k] for k in unique_coord.get((round_x_pixel[j],round_y_pixel[j]))])

        im = Image.fromarray(np.uint8(cm.Greys(imgM)*255))

        #np.save("UMAP_images/mrna515_0.1_5/data_px/" + datafile + "_" + patient_id[i], imgM)
        
        #plt.figure(figsize=(5,10))
        plt.axis('off')
        plt.imshow(imgM, cmap='BuPu',vmin=0, vmax=1)

        if status[i] == 0:
            np.save("tsne_images/mrna515_tsne_6_cosine_100itr/data_px/class_0/" + patient_id[i] + '_' + datafile, imgM)
            im.save('tsne_images/mrna515_tsne_6_cosine_100itr/class_0/' + patient_id[i] + '_' + datafile + '.png')
            #plt.savefig('tsne_images/mrna515_tsne_6_cosine/class_0/_' + patient_id[i] + '.png', bbox_inches='tight', format = "png", dpi=96)
            #plt.savefig('tsne_images/mrna515/class_0/_' + patient_id[i] + '.png', bbox_inches='tight', format = "png", dpi=96)
            
        else:
            np.save("tsne_images/mrna515_tsne_6_cosine_100itr/data_px/class_1/" + patient_id[i] + '_' + datafile, imgM)
            im.save('tsne_images/mrna515_tsne_6_cosine_100itr/class_1/' + patient_id[i] + '_' + datafile + '.png')
            #plt.savefig('tsne_images/mrna515_tsne_6_cosine/class_1/_' + patient_id[i] + '.png', bbox_inches='tight', format = "png", dpi=96)

        plt.close()

    print(datafile, "Done!")

In [ ]:
#for cancer genes only image
color_idx = pd.read_csv("color_idx.csv")
final_df = pd.read_csv("plots_515/tsne/data_finaldf/tsne_6___cosine.csv")
#pix_subset

final_df = final_df.drop('c', axis =1)
final_df = pd.concat([final_df, color_idx], axis=1)
final_df.columns = ['a', 'b', 'c']

df2= pd.concat([pix_subset, final_df['c']], axis = 1)
df2 = df2.loc[df2['c']>=2] 
df3 = df2.drop(['c'], axis = 1)
# df3_a = df3['pix-tsne-2d-one'].values
# df3_b = df3['pix-tsne-2d-two'].values

# #final_df.columns = ['a', 'b', 'c']
# df = final_df.loc[final_df['c']>=2] 

# df2= pd.concat([pix_subset, df['c']], axis = 1)
# df2 = df2.loc[df2['c']>=2] 

# df3 = df2.drop(['c'], axis = 1)
df3_a = df3['pix-tsne-2d-one'].values
df3_b = df3['pix-tsne-2d-two'].values

In [ ]:
#df3.reset_index(level=0, inplace=True)
idx_cancer_genes = df3['index'].values
np.save('analysis/idx_C_pix.npy', idx_cancer_genes)

In [ ]:
idx_cancer_genes

In [ ]:
#*******************for image with cancer genes********************
pixel_set_C = []
for _ in range(len(df3)):
    pixel_set_C.append((df3_a[_],df3_b[_]))

unique_coord_C = {}
for i in range(len(pixel_set_C)):
    unique_coord_C[pixel_set_C[i]] = list_duplicates_of(pixel_set_C,pixel_set_C[i])

# for i in range(norm_data.shape[0]):
#     sample_no = i
#     imgC = np.zeros((new_A+1,new_B+1))
#     #fvec = norm_data.iloc[sample_no,:]

#     #for j in range(norm_data.shape[1]):
#     for j in range(len(df3_a)):
#         imgC[df3_a[j], df3_b[j]] = 255

#         #if j%5 == 0:
#             #plt.imshow(imgC, cmap='BuPu')
#             #plt.savefig('image_' + str(j) + '.png', bbox_inches='tight', format = "png", dpi=96)
#         #print(df3_a[j], df3_b[j],j)

#     #np.save("UMAP_images/mrna515_0.1_5/data_px/" + datafile + "_" + patient_id[i], imgM)
#     #np.save("tsne_images/mrna515/data_px/_" + patient_id[i], imgC)
#     #plt.figure(figsize=(5,10))
#     #plt.axis('off')
#     #plt.imshow(imgC, cmap='BuPu',vmin=0, vmax=1)
#     plt.imshow(imgC, cmap='BuPu')
#     plt.savefig('image_final_cancer.png', bbox_inches='tight', format = "png", dpi=96)

#     # if status[i] == 0:
#     #     plt.savefig('tsne_images/mrna515/class_0/_' + patient_id[i] + '.png', bbox_inches='tight', format = "png", dpi=96)
#     #     #plt.savefig('tsne_images/mrna515/class_0/_' + patient_id[i] + '.png', bbox_inches='tight', format = "png", dpi=96)
        
#     # else:
#     #     plt.savefig('tsne_images/mrna515/class_1/_' + patient_id[i] + '.png', bbox_inches='tight', format = "png", dpi=96)

#     # plt.close()

# # print(patient_id[i], "Done!")

In [ ]:
#To extract location of cancer genes and all the genes in 122x122 pixelsize image
len(unique_coord_C)
np.save('analysis/genlist_pix.npy', unique_coord)
np.save('analysis/genlist_C_pix.npy', unique_coord_C)

In [ ]:
df_subset

In [6]:
#df_subset = pd.read_csv("plots_515/tsne_6_cosine/data_finaldf/" + datafile) #tsne output
#df_subset = pd.read_csv("tsne_seed22/mirna/final_df_cosine/tsne_8___cosine.csv") #tsne output
df_subset = pd.read_csv("umap_seed22/mirna/final_df_cosine/umap_2___0.6.csv") #umap output
#df_subset = pd.read_csv("plots_515/tsne/data_finaldf/tsne_6___cosine.csv") # tsne_out
#df_subset = df_subset[['a', 'b']]
#print("Datafile:", datafile)

def minimum_bounding_rectangle(points):
    """
    Find the smallest bounding rectangle for a set of points.
    Returns a set of points representing the corners of the bounding box.

    :param points: an nx2 matrix of coordinates
    :rval: an nx2 matrix of coordinates
    """
    from scipy.ndimage.interpolation import rotate
    pi2 = np.pi/2.

    # get the convex hull for the points
    hull_points = points[ConvexHull(points).vertices]

    # calculate edge angles
    edges = np.zeros((len(hull_points)-1, 2))
    edges = hull_points[1:] - hull_points[:-1]

    angles = np.zeros((len(edges)))
    angles = np.arctan2(edges[:, 1], edges[:, 0])

    angles = np.abs(np.mod(angles, pi2))
    angles = np.unique(angles)

    # find rotation matrices
    # XXX both work
    rotations = np.vstack([
        np.cos(angles),
        np.cos(angles-pi2),
        np.cos(angles+pi2),
        np.cos(angles)]).T
    rotations = rotations.reshape((-1, 2, 2))

    # apply rotations to the hull
    rot_points = np.dot(rotations, hull_points.T)

    # find the bounding points
    min_x = np.nanmin(rot_points[:, 0], axis=1)
    max_x = np.nanmax(rot_points[:, 0], axis=1)
    min_y = np.nanmin(rot_points[:, 1], axis=1)
    max_y = np.nanmax(rot_points[:, 1], axis=1)

    # find the box with the best area
    areas = (max_x - min_x) * (max_y - min_y)
    best_idx = np.argmin(areas)

    # return the best box
    x1 = max_x[best_idx]
    x2 = min_x[best_idx]
    y1 = max_y[best_idx]
    y2 = min_y[best_idx]
    r = rotations[best_idx]

    rval = np.zeros((4, 2))
    rval[0] = np.dot([x1, y2], r)
    rval[1] = np.dot([x2, y2], r)
    rval[2] = np.dot([x2, y1], r)
    rval[3] = np.dot([x1, y1], r)

    return rval

mbr = minimum_bounding_rectangle(df_subset.values)

# Calculate tilit of bounding box
y2,y1,x2,x1 = mbr[0][1],mbr[1][1],mbr[0][0],mbr[1][0]
theta = (y2-y1)/(x2-x1)
angle  = math.degrees(np.arctan(theta))
angle  = np.arctan(theta)
r_matrix = np.asarray([[math.cos(angle),-1*math.sin(angle)],[math.sin(angle),math.cos(angle)]])

zrect = np.matmul(mbr,r_matrix)  
z = np.asarray(np.matmul(df_subset,r_matrix))
rz_subset  = pd.DataFrame()
rz_subset['rot-tsne-2d-one'] = z[:,0]
rz_subset['rot-tsne-2d-two'] = z[:,1]

rz_subset.to_csv('umap_seed22/mirna/deepinsight/mirna_umap_2___0.6_rotated.csv')

z_dist = distance.cdist(z, z, 'euclidean')
min_z_dist = z_dist[z_dist>0].min()

rec_x_axis,rec_y_axis = abs(zrect[0][0] - zrect[1][0]),abs(zrect[1][1] - zrect[2][1])

precision_old = math.sqrt(2)
A = math.ceil((rec_x_axis*precision_old)/min_z_dist)
B = math.ceil((rec_y_axis*precision_old)/min_z_dist)

max_pix_size = 30
if (max([A,B]) > max_pix_size):
    precision = (precision_old*max_pix_size/max([A,B]))
    A = math.ceil((rec_x_axis*precision)/min_z_dist)
    B = math.ceil((rec_y_axis*precision)/min_z_dist)

x_coord,y_coord = rz_subset.iloc[:,0].values,rz_subset.iloc[:,1].values
x_min,y_min,x_max,y_max = min(x_coord),min(y_coord),max(x_coord),max(x_coord)
x_pixel = (1 + (A*(x_coord - x_min))/(x_max - x_min))
y_pixel = (1 + (B*(y_coord - y_min))/(y_max - y_min))
round_x_pixel = np.array([int(np.round(x)) for x in x_pixel])
round_y_pixel = np.array([int(np.round(y)) for y in y_pixel])

pix_subset  = pd.DataFrame()
pix_subset['pix-tsne-2d-one'] = round_x_pixel
pix_subset['pix-tsne-2d-two'] = round_y_pixel
pix_subset.to_csv('umap_seed22/mirna/deepinsight/mirna_umap_2___0.6_pix_coord.csv')

#function to find overlapping points
def list_duplicates_of(seq,item):
    start_at = -1
    locs = []
    while True:
        try:
            loc = seq.index(item,start_at+1)
        except ValueError:
            break
        else:
            locs.append(loc)
            start_at = loc
    return locs

#********************************For full image**************************
pixel_set = []
for _ in range(len(round_x_pixel)):
    pixel_set.append((round_x_pixel[_],round_y_pixel[_]))

unique_coord = {}
for i in range(len(pixel_set)):
    unique_coord[pixel_set[i]] = list_duplicates_of(pixel_set,pixel_set[i])

clean_data = data.iloc[:,1:].T
mmscaler = MinMaxScaler()
#mmscaler.fit(clean_data)
norm_data = pd.DataFrame(mmscaler.fit_transform(clean_data))
sample_no = 0
fvec = norm_data.iloc[sample_no,:]

new_A = max(round_x_pixel)
new_B = max(round_y_pixel)

im_size =  max(new_A, new_B)

for i in range(norm_data.shape[0]):
    sample_no = i
    imgM = np.zeros((im_size+1,im_size+1))
    fvec = norm_data.iloc[sample_no,:]
    for j in range(norm_data.shape[1]):
        imgM[round_x_pixel[j]-1,round_y_pixel[j]-1] = mean([fvec[k] for k in unique_coord.get((round_x_pixel[j],round_y_pixel[j]))])

    im = Image.fromarray(np.uint8(cm.Greys(imgM)*255))

    #np.save("UMAP_images/mrna515_0.1_5/data_px/" + datafile + "_" + patient_id[i], imgM)
    
    #plt.figure(figsize=(5,10))
    plt.axis('off')
    plt.imshow(imgM, cmap='BuPu',vmin=0, vmax=1)

    np.save("umap_seed22/mirna/deepinsight/mirna_umap_2___0.6/data_px/" + patient_id[i], imgM)
    im.save('umap_seed22/mirna/deepinsight/mirna_umap_2___0.6/' + patient_id[i] + '.png')

    # if status[i] == 0:
    #     np.save("tsne_515_seed22/deepinsight_outputs/mrna515_tsne_5_cosine/data_px/class_0/" + patient_id[i], imgM)
    #     im.save('tsne_515_seed22/deepinsight_outputs/mrna515_tsne_5_cosine/class_0/' + patient_id[i] + '.png')
    #     #plt.savefig('tsne_images/mrna515_tsne_6_cosine/class_0/_' + patient_id[i] + '.png', bbox_inches='tight', format = "png", dpi=96)
    #     #plt.savefig('tsne_images/mrna515/class_0/_' + patient_id[i] + '.png', bbox_inches='tight', format = "png", dpi=96)
        
    # elif status[i] == 1:
    #     np.save("tsne_515_seed22/deepinsight_outputs/mrna515_tsne_5_cosine/data_px/class_1/" + patient_id[i], imgM)
    #     im.save('tsne_515_seed22/deepinsight_outputs/mrna515_tsne_5_cosine/class_1/' + patient_id[i] + '.png')
    #     #plt.savefig('tsne_images/mrna515_tsne_6_cosine/class_1/_' + patient_id[i] + '.png', bbox_inches='tight', format = "png", dpi=96)

    plt.close()

    print(i, "Done!")

0 Done!
1 Done!
2 Done!
3 Done!
4 Done!
5 Done!
6 Done!
7 Done!
8 Done!
9 Done!
10 Done!
11 Done!
12 Done!
13 Done!
14 Done!
15 Done!
16 Done!
17 Done!
18 Done!
19 Done!
20 Done!
21 Done!
22 Done!
23 Done!
24 Done!
25 Done!
26 Done!
27 Done!
28 Done!
29 Done!
30 Done!
31 Done!
32 Done!
33 Done!
34 Done!
35 Done!
36 Done!
37 Done!
38 Done!
39 Done!
40 Done!
41 Done!
42 Done!
43 Done!
44 Done!
45 Done!
46 Done!
47 Done!
48 Done!
49 Done!
50 Done!
51 Done!
52 Done!
53 Done!
54 Done!
55 Done!
56 Done!
57 Done!
58 Done!
59 Done!
60 Done!
61 Done!
62 Done!
63 Done!
64 Done!
65 Done!
66 Done!
67 Done!
68 Done!
69 Done!
70 Done!
71 Done!
72 Done!
73 Done!
74 Done!
75 Done!
76 Done!
77 Done!
78 Done!
79 Done!
80 Done!
81 Done!
82 Done!
83 Done!
84 Done!
85 Done!
86 Done!
87 Done!
88 Done!
89 Done!
90 Done!
91 Done!
92 Done!
93 Done!
94 Done!
95 Done!
96 Done!
97 Done!
98 Done!
99 Done!
100 Done!
101 Done!
102 Done!
103 Done!
104 Done!
105 Done!
106 Done!
107 Done!
108 Done!
109 Done!
110 Done!


In [ ]:
unique_coord
# Save
#np.save('genlist_pix.npy', unique_coord)

In [ ]:
import imageio
import os
images = []
for filename in os.listdir("UMAP_results"):

    if filename.endswith(('.png')):
        images.append(imageio.imread("UMAP_results/" + filename))
imageio.mimsave('movie.gif', images)